In [1]:
!pip -q install -U iterative-stratification

In [2]:
%%writefile convert_npy_to_png.py
#src/features/convert_npy_to_png.py
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
from PIL import Image

def normalize_to_uint8(img):
    img = np.clip(img, 0, 10000)
    img = img / 10000.0
    img = (img * 255).astype(np.uint8)
    return img

def convert_npy_to_png(npy_folder, output_folder, bands=[3,2,1]):
    npy_folder = Path(npy_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    for filename in tqdm(os.listdir(npy_folder), desc=f"Converting {npy_folder.name}"):
        if filename.endswith(".npy"):
            path = npy_folder / filename
            data = np.load(path)
            rgb = data[:, :, bands]
            rgb = normalize_to_uint8(rgb)
            image_pil = Image.fromarray(rgb)
            out_path = output_folder / filename.replace(".npy", ".png")
            image_pil.save(out_path)

Writing convert_npy_to_png.py


In [3]:
%%writefile train_efficientnet.py
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import timm
from torch.nn.functional import sigmoid
from config_efficientnet import CFG, seed_everything
import random
import warnings
warnings.filterwarnings('ignore')

seed_everything(CFG.seed)

# === Dataset ===
class SentinelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = row['path']
        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            raise FileNotFoundError(f"Could not read image at {img_path}")
        if image.ndim == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:
            image = image[:, :, :3]
        if self.transform:
            image = self.transform(image=image)["image"]
        if self.to_train:
            target = torch.tensor(row["label"], dtype=torch.float32)
            return image, target
        return image

# === Model ===
class EfficientNetV2Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("efficientnetv2_rw_m", pretrained=True)
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
        self.classifier = nn.Linear(in_features, 1)
    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)

# === Transforms ===
train_transforms = A.Compose([
    A.Resize(CFG.img_size, CFG.img_size),
    A.Rotate(limit=30, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=5, p=0.3),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])
test_transforms = A.Compose([
    A.Resize(CFG.img_size, CFG.img_size),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# === Training ===
def train_one_fold(fold, train_loader, val_loader, model_dir):
    seed_everything(42)
    model = EfficientNetV2Classifier().cuda()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs)  # <-- FIX: match script
    scaler = torch.amp.GradScaler('cuda')
    best_loss = float("inf")
    fold_path = Path(model_dir) / f"fold_{fold+1}"
    fold_path.mkdir(parents=True, exist_ok=True)
    best_model_path = fold_path / "best.pt"

    for epoch in range(CFG.epochs):
        model.train()
        epoch_loss = 0.0
        for images, targets in tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{CFG.epochs} - Train"):
            images, targets = images.cuda(), targets.cuda().unsqueeze(1).float()
            optimizer.zero_grad()
            with torch.amp.autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.inference_mode():
            for images, targets in val_loader:
                images, targets = images.cuda(), targets.cuda().unsqueeze(1).float()
                outputs = model(images)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f"Fold {fold+1} Epoch {epoch+1}: Train Loss {epoch_loss/len(train_loader):.4f}, Val Loss {val_loss:.4f}")
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
        scheduler.step()
    return best_model_path

# === Main Training ===
def train_efficientnet(train_csv, test_csv, train_imgs, test_imgs, model_dir):
    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)
    train_df = train_df.groupby("ID").sum().reset_index()[["ID", "label"]]
    train_df["path"] = train_imgs + "/" + train_df["ID"] + ".png"
    test_df["path"] = test_imgs + "/" + test_df["ID"] + ".png"
    skf = StratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)
    train_df["stratify_label"] = train_df[["label"]].sum(axis=1)
    for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
        train_df.loc[valid_idx, "fold"] = fold
    for fold in range(CFG.folds):
        train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
        valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)
        train_loader = DataLoader(SentinelDataset(train_data, transform=train_transforms), batch_size=CFG.batch_size, shuffle=True, num_workers=os.cpu_count())
        val_loader = DataLoader(SentinelDataset(valid_data, transform=test_transforms), batch_size=CFG.batch_size, shuffle=False)
        train_one_fold(fold, train_loader, val_loader, model_dir)

# === Inference (Ensemble of folds) ===
def infer_efficientnet(test_csv, test_imgs, model_dir, out_csv):
    test_df = pd.read_csv(test_csv)
    test_df["path"] = test_imgs + "/" + test_df["ID"] + ".png"
    test_loader = DataLoader(SentinelDataset(test_df, transform=test_transforms, to_train=False), batch_size=CFG.batch_size, shuffle=False)
    probs = np.zeros(len(test_df))
    model = EfficientNetV2Classifier().cuda()
    for fold in range(CFG.folds):
        fold_model = Path(model_dir) / f"fold_{fold+1}" / "best.pt"
        model.load_state_dict(torch.load(fold_model))
        model.eval()
        fold_preds = []
        with torch.inference_mode():
            for images in tqdm(test_loader, desc=f"Inference Fold {fold+1}"):
                images = images.cuda()
                logits = model(images)
                preds = sigmoid(logits).squeeze(1).cpu().numpy()
                fold_preds.append(preds)
        probs += np.concatenate(fold_preds)
    probs /= CFG.folds
    pd.DataFrame({"ID": test_df["ID"], "probability": probs}).to_csv(out_csv, index=False)
    print(f"✅ Predictions saved to {out_csv}")

Writing train_efficientnet.py


In [4]:
%%writefile config_efficientnet.py
import os
import random
import numpy as np
import torch

class CFG:
    seed = 42
    folds = 5
    epochs = 20
    img_size = 320
    batch_size = 16
    lr = 1e-4

def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)

Writing config_efficientnet.py


In [ ]:
%%writefile main_efficientnet.py

import argparse
from convert_npy_to_png import convert_npy_to_png
from pathlib import Path
from train_efficientnet import train_efficientnet, infer_efficientnet

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", choices=["train", "infer"], required=True)
    parser.add_argument("--train_csv", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Train.csv")
    parser.add_argument("--test_csv", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Test.csv")
    parser.add_argument("--train_npy_dir", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data")
    parser.add_argument("--test_npy_dir", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/test_data/test_data")
    parser.add_argument("--image_dir", type=str, default="train_data_sentinel_png")
    parser.add_argument("--test_dir", type=str, default="test_data_sentinel_png")
    parser.add_argument("--model_dir", type=str, default="models/efficientnet")
    parser.add_argument("--output_path", type=str, default="efficientnet_submission_probs.csv")
    args = parser.parse_args()

    if args.mode == "train":
        # Step 1: Convert npy → png (if not already done)
        if not Path(args.image_dir).exists() or len(list(Path(args.image_dir).glob("*.png"))) == 0:
            print(f"⚙️ Converting training npy files from {args.train_npy_dir} to PNG at {args.image_dir}")
            convert_npy_to_png(args.train_npy_dir, args.image_dir)
        if not Path(args.test_dir).exists() or len(list(Path(args.test_dir).glob("*.png"))) == 0:
            print(f"⚙️ Converting test npy files from {args.test_npy_dir} to PNG at {args.test_dir}")
            convert_npy_to_png(args.test_npy_dir, args.test_dir)

        # Step 2: Train YOLO models
        train_efficientnet(args.train_csv, args.test_csv, args.image_dir, args.test_dir, args.model_dir)

    elif args.mode == "infer":
        infer_efficientnet(args.test_csv,args.test_dir, args.model_dir, args.output_path)

Writing main_efficientnet.py


In [ ]:
# Train
!python main_efficientnet.py --mode train

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
⚙️ Converting training npy files from /kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data to PNG at train_data_sentinel_png
Converting train_data: 100%|████████████████| 7147/7147 [01:21<00:00, 87.65it/s]
⚙️ Converting test npy files from /kaggle/input/slideandseekclasificationlandslidedetectiondataset/test_data/test_data to PNG at test_data_sentinel_png
Converting test_data: 100%|█████████████████| 5397/5397 [01:00<00:00, 89.40it/s]
model.safetensors: 100%|██████████████████████| 214M/214M [00:00<00:00, 232MB/s]
Fold 1 Epoch 1/20 - Train: 100%|██████████████| 358/358 [02:05<00:00,  2.84it/s]
Fold 1 Epoch 1: Train Loss 0.3099, Val Loss 0.

In [ ]:
# Infer
!python main_efficientnet.py --mode infer